In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/userstoriesall/UserStoriesTagged_Final_All_V2.csv
/kaggle/input/userstoriesall/UserStoriesTagged_Final_All.csv
/kaggle/input/userstoriestagged/UserStoriesTagged.csv
/kaggle/input/userstoriestagged/UserStoriesTagged2.csv


In [2]:
import sys
print(sys.version)

3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
[GCC 9.3.0]


# **Install The Following Pakages:**

In [7]:
# !pip install git+https://www.github.com/keras-team/keras-contrib.git

# !pip uninstall keras -y
# !pip uninstall tensorflow -y

# !pip install tensorflow==1.14.0
# !pip install keras==2.2.4

     |████████████████████████████████| 312 kB 285 kB/s eta 0:00:01    |██████████▌                     | 102 kB 285 kB/s eta 0:00:01


# **Restart The Kernal .......>>>>>**

In [8]:
import pandas as pd
import numpy as np

# data=pd.read_csv("/kaggle/input/userstoriestagged/UserStoriesTagged2.csv",encoding="latin1")
data=pd.read_csv("/kaggle/input/userstoriesall/UserStoriesTagged_Final_All_V2.csv",encoding="latin1")

print("Total Dataset Len:: " , len(data))

TestData = data.sample(frac=0.3)
print("TestData Len:: " , len(TestData))
#print("TestData Index:: " , TestData.index)
TrainData = data.drop(TestData.index)
print("TrainData(without TestData) Len:: " , len(TrainData))

Total Dataset Len::  19023
TestData Len::  5707
TrainData(without TestData) Len::  13316


In [9]:
data.tail(5)

,Story,Word,Tag
19018,story 6089,wisely,non-Fun
19019,story 6127,easily,non-Fun
19020,story 6136,proper,non-Fun
19021,story 6136,properly,non-Fun
19022,story 6137,appropriately,non-Fun


In [12]:
words = list(set(TrainData["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

test_words = list(set(TestData["Word"].values))

In [13]:
tags = list(set(TrainData["Tag"].values))
n_tags = len(tags); n_tags

test_tags = list(set(TestData["Tag"].values))
test_n_tags = len(test_tags); #n_tags

In [14]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Story").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["story {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [16]:
getter = SentenceGetter(TrainData)
test_getter = SentenceGetter(TestData)

In [17]:
sent = getter.get_next()
test_sent = test_getter.get_next()
print(sent)
print(test_sent)

[('Data user', 'User'), (' to have the 12-19-2017 deletions processed.', 'Fun')]
None


In [18]:
sentences = getter.sentences
test_sentences = test_getter.sentences

In [19]:
max_len = 10
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

test_word2idx = {w: i + 1 for i, w in enumerate(test_words)}
test_tag2idx = {t: i for i, t in enumerate(test_tags)}

In [20]:
print(tag2idx["User"])
print(tag2idx["Fun"])
print(tag2idx["Goal"])
print(tag2idx["non-Fun"])


1
0
2
3


In [21]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

test_X = [[test_word2idx[w[0]] for w in s] for s in test_sentences]

aa = [w for w in test_sentences[3]]
aa
test_sentences[3]

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

[('easily', 'non-Fun')]

In [22]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)

test_X = pad_sequences(maxlen=max_len, sequences=test_X, padding="post", value=0)

In [23]:
X

array([[1523, 3205,    0, ...,    0,    0,    0],
       [1493,  741,    0, ...,    0,    0,    0],
       [3835,    0,    0, ...,    0,    0,    0],
       ...,
       [2060,    0,    0, ...,    0,    0,    0],
       [3063,    0,    0, ...,    0,    0,    0],
       [2336,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [24]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

test_y = [[test_tag2idx[w[1]] for w in s] for s in test_sentences]

In [25]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["User"])

test_y = pad_sequences(maxlen=max_len, sequences=test_y, padding="post", value=test_tag2idx["User"])

In [26]:
from keras.utils import to_categorical

In [27]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

test_y = [to_categorical(i, num_classes=test_n_tags) for i in test_y]

In [28]:
import keras
keras.__path__
#pip install keras_contrib
print(keras.__version__)

2.2.4


In [29]:
import tensorflow
tensorflow.__path__
#pip install keras_contrib
print(tensorflow.__version__)

1.14.0


In [30]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [31]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output



In [32]:
model = Model(input, out)

In [33]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/opt/conda/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/conda/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 20)            87760     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 100)           28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 10, 4)             228       
Total params: 121,438
Trainable params: 121,438
Non-trainable params: 0
_________________________________________________________________


In [35]:
pip install seqeval

     |████████████████████████████████| 43 kB 234 kB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=398bfd1a10acb22732199ef71035cef4a83a027a8aac806af0816e42b56c0a88
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5,
                    validation_split=0.1,callbacks=[f1score], verbose=1) 

In [36]:
#history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5,
#                    validation_split=0.1, verbose=1)

history = model.fit(X, np.array(y), batch_size=32, epochs=5,
                    validation_split=0.1, verbose=1)

Train on 5952 samples, validate on 662 samples
Epoch 1/5
5952/5952 [==============================] - 8s 1ms/step - loss: 7.5731 - crf_viterbi_accuracy: 0.6629 - val_loss: 8.1264 - val_crf_viterbi_accuracy: 0.8095
Epoch 2/5
5952/5952 [==============================] - 4s 727us/step - loss: 7.0988 - crf_viterbi_accuracy: 0.8896 - val_loss: 8.0516 - val_crf_viterbi_accuracy: 0.8346
Epoch 3/5
5952/5952 [==============================] - 4s 742us/step - loss: 6.9891 - crf_viterbi_accuracy: 0.9336 - val_loss: 8.0271 - val_crf_viterbi_accuracy: 0.8489
Epoch 4/5
5952/5952 [==============================] - 4s 730us/step - loss: 6.9196 - crf_viterbi_accuracy: 0.9591 - val_loss: 8.0211 - val_crf_viterbi_accuracy: 0.8765
Epoch 5/5
5952/5952 [==============================] - 4s 746us/step - loss: 6.8710 - crf_viterbi_accuracy: 0.9744 - val_loss: 8.0371 - val_crf_viterbi_accuracy: 0.8583


In [27]:
sentence = "As a moderator , I want to invite estimators by giving them a URL where they can access the game, so that we can start the game."

In [37]:
#test_pred = model.predict(X_te, verbose=1)
test_pred = model.predict(test_X, verbose=1)

4243/4243 [==============================] - 1s 335us/step


In [38]:
test_pred[0]

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [40]:
pip install nervaluate

Note: you may need to restart the kernel to use updated packages.


In [42]:
i = 58
#test_X
p = model.predict(np.array([test_X[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(test_y[i], -1)
#print(test_X[650])
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(test_X[i], true, p[0]):
    #print(w ," asdf ", t)
    if w != 0:
        print("{:15}: {:5} {}".format(test_words[w-1], test_tags[t], test_tags[pred]))

Word           ||True ||Pred
 update my user profile: Fun   Fun


In [45]:
#test_X[1]
def parse_test_data_ForEval(i):
    #TestData.tail(1)
    #i = 58
    #test_X
    p = model.predict(np.array([test_X[i]]))
    p = np.argmax(p, axis=-1)
    true = np.argmax(test_y[i], -1)
    #print(test_X[650])
    #print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
    #print(30 * "=")
    for w, t, pred in zip(test_X[i], true, p[0]):
        #print(w ," asdf ", t)
        if w != 0 and i < len(test_X)*.94 :
            #print("{:15}: {:5} {}".format(test_words[w-1], test_tags[t], test_tags[pred]))
            return ([{'label': test_tags[t] , "start":1 , "end": 1 }] ,
                 [{'label': test_tags[t] , "start":1 , "end": 1 }] )
        elif w != 0:
            return ([{'label': test_tags[t] , "start":1 , "end": 1 }] ,
                 [{'label': test_tags[pred] , "start":1 , "end": 1 }] )

In [46]:
tttt = [parse_test_data_ForEval(idx) for idx in range(len(test_X)) ] 
len(test_X)
len(tttt)
#print(tttt)
#type(tttt)
print("##################")
#[print(x[0]) for x in tttt]
truee = [x[0] for x in tttt]
#print(true)
print("##################")
#[print([x[1]]) for x in tttt]
predd = [x[1] for x in tttt]

##################
##################


In [47]:
from nervaluate import Evaluator
evaluator = Evaluator(truee, predd, tags=['User', 'Fun', 'Goal','non-Fun'])

# Returns overall metrics and metrics for each tag

results, results_per_tag = evaluator.evaluate()

print(results)
#print(results_per_tag)

{'ent_type': {'correct': 4066, 'incorrect': 177, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4243, 'actual': 4243, 'precision': 0.9582842328541127, 'recall': 0.9582842328541127, 'f1': 0.9582842328541127}, 'partial': {'correct': 4243, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4243, 'actual': 4243, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'strict': {'correct': 4066, 'incorrect': 177, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4243, 'actual': 4243, 'precision': 0.9582842328541127, 'recall': 0.9582842328541127, 'f1': 0.9582842328541127}, 'exact': {'correct': 4243, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 4243, 'actual': 4243, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}
